In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.layers as L
import pandas as pd
import cv2
from tqdm import tqdm, tqdm_notebook
import os

%matplotlib inline

Using TensorFlow backend.


**map index to label**

In [2]:
data = pd.read_csv('data/train_gt.csv')
data.head()

,filename,classnum
0,0000.avi,56
1,0001.avi,51
2,0002.avi,64
3,0003.avi,23
4,0004.avi,41


In [3]:
class_num = []
file_name = []
for row in data.iterrows():
    name, label = row[1]['filename'], row[1]['classnum']
    idx = name.split('.')[0]
    file_name.append(idx)
    class_num.append(int(label))
class_num = np.array(class_num)
file_name = np.array(file_name)

In [4]:
class_num[:10], file_name[:10]

(array([56, 51, 64, 23, 41, 69, 11, 34, 47, 16]),
 array(['0000', '0001', '0002', '0003', '0004', '0005', '0006', '0007',
        '0008', '0009'], dtype='<U4'))

In [5]:
class_num.min(), class_num.max()

(0, 100)

In [6]:
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.cudnn_recurrent import CuDNNLSTM
from keras.layers.pooling import GlobalAveragePooling1D, GlobalAveragePooling2D
from keras.layers.merge import Average
from keras.models import Sequential, Model
from keras.layers.wrappers import TimeDistributed
from keras.layers import InputLayer, Input
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input
from keras.utils import np_utils
from keras import backend as K
from keras.utils import multi_gpu_model

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(file_name, class_num)
X_train.shape, y_train.shape, X_train[:5], y_train[:5]

((7097,),
 (7097,),
 array(['1717', '5174', '2784', '2820', '1936'], dtype='<U4'),
 array([84, 49, 30,  6,  0]))

In [9]:
y_train = np_utils.to_categorical(y_train, class_num.max() + 1)
y_test = np_utils.to_categorical(y_test, class_num.max() + 1)

In [10]:
y_train.shape

(7097, 101)

In [11]:
num_frames = []
for name in os.listdir('data/train_converted/imgs/'):
    num_frames.append(len(os.listdir('data/train_converted/imgs/' + name)))
np.max(num_frames), np.min(num_frames)

(1776, 30)

In [16]:
a = np.zeros((4, 30, 240, 320, 3))
a[1:3, [1,2,3,4]].shape

(2, 4, 240, 320, 3)

In [20]:
X_train.shape

(7097,)

In [59]:
NUM_FRAMES = 10
BATCH_SIZE = 32
IMAGE_SHAPE = (240, 320, 3)
TENSOR_SHAPE = (NUM_FRAMES, ) + IMAGE_SHAPE
TRAIN_IMGS_PATH = 'data/train_converted/imgs/'
TRAIN_FLOW_PATH = 'data/train_converted/flow/'

In [48]:
def get_train_generator(data, label, batch_size=32, circle=True, shuffle_data=True, shuffle_frame=True):
    while True:
        indices = np.arange(len(data))
        if shuffle_data:
            indices = np.random.permutation(indices)
        for start in range(0, len(data), batch_size):
            batch_idx = indices[start : start + batch_size]
            y_batch = label[batch_idx]
            X_batch = np.zeros((y_batch.shape[0],) + TENSOR_SHAPE)
            for i, name in enumerate(data[batch_idx]):
                for k in range(NUM_FRAMES):
                    if not shuffle_frame:
                        start_frame = 0
                    else:
                        start_frame = np.random.randint(int(0.5 * len(os.listdir(TRAIN_IMGS_PATH + name))))
                    img = cv2.imread(TRAIN_IMGS_PATH + '{}/{}.jpg'.format(name, k + start_frame))
                    X_batch[i, k] = preprocess_input(img)
            yield X_batch, y_batch
        if not circle:
            break

In [60]:
def get_train_flow_generator(data, label, batch_size=32, circle=True, shuffle_data=True, shuffle_frame=True):
    while True:
        indices = np.arange(len(data))
        if shuffle_data:
            indices = np.random.permutation(indices)
        for start in range(0, len(data), batch_size):
            batch_idx = indices[start : start + batch_size]
            y_batch = label[batch_idx]
            X_batch = np.zeros((y_batch.shape[0],) + TENSOR_SHAPE)
            for i, name in enumerate(data[batch_idx]):
                for k in range(NUM_FRAMES):
                    if not shuffle_frame:
                        start = 1
                    else:
                        start = np.random.randint(1, int(0.5 * len(os.listdir(TRAIN_FLOW_PATH + name))))
                    img = cv2.imread(TRAIN_FLOW_PATH + '{}/{}.jpg'.format(name, k + start))
                    X_batch[i, k] = preprocess_input(img)
            yield X_batch, y_batch
        if not circle:
            break

In [87]:
def get_eval_generator(names, batch_size=32):
    for start in range(0, len(names), batch_size):
        sub_names = names[start : start + batch_size]
        X_batch = np.zeros((len(sub_names),) + TENSOR_SHAPE)
        for i, name in enumerate(sub_names):
            for k in range(NUM_FRAMES):
                start_frame = np.random.randint(int(0.5 * len(os.listdir('data/test_converted/imgs/' + name))))
                img = cv2.imread('data/test_converted/imgs/{}/{}.jpg'.format(name, k + start_frame))
                X_batch[i, k] = preprocess_input(img)
        yield X_batch

In [28]:
global MODEL
MODEL = Xception(include_top=False, weights='imagenet')
global GRAPH
GRAPH = tf.get_default_graph()

In [29]:
def get_rgb_model(lstm_size):
    base_model = TimeDistributed(MODEL)
    base_model(Input(shape=TENSOR_SHAPE))
    
    x = base_model.output
    x = TimeDistributed(GlobalAveragePooling2D())(x)
    x = CuDNNLSTM(lstm_size)(x)
    predictions = Dense(101, activation='softmax')(x)

    with tf.device('/cpu:0'):
        model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layer.layers:
        layer.trainable = False

    parallel_model = multi_gpu_model(model)
    parallel_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return parallel_model, model

In [30]:
super_model, model = get_rgb_model(512)

In [78]:
train_generator = get_train_generator(X_train, y_train, batch_size=BATCH_SIZE, circle=True, shuffle_data=True, shuffle_frame=True)
valid_generator = get_train_generator(X_test, y_test, batch_size=BATCH_SIZE, circle=True, shuffle_data=False, shuffle_frame=False)

In [50]:
X_test.shape, y_test.shape

((2366,), (2366, 101))

In [79]:
valid_full_path_steps = np.ceil(len(X_test) / BATCH_SIZE)

In [80]:
super_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
super_model.fit_generator(train_generator, steps_per_epoch=100, epochs=10, validation_data=valid_generator, 
                          validation_steps=valid_full_path_steps, max_queue_size=30, workers=3, 
                          use_multiprocessing=True)

Epoch 1/10


/home/ya-philya/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


100/100 [==============================] - 243s 2s/step - loss: 3.4939 - acc: 0.2591 - val_loss: 2.9201 - val_acc: 0.3079
Epoch 2/10
100/100 [==============================] - 231s 2s/step - loss: 0.8737 - acc: 0.7909 - val_loss: 1.6641 - val_acc: 0.5621
Epoch 3/10
100/100 [==============================] - 226s 2s/step - loss: 0.5894 - acc: 0.8494 - val_loss: 1.5734 - val_acc: 0.5940
Epoch 4/10
100/100 [==============================] - 219s 2s/step - loss: 0.4970 - acc: 0.8759 - val_loss: 1.4500 - val_acc: 0.6090
Epoch 5/10
100/100 [==============================] - 216s 2s/step - loss: 0.4823 - acc: 0.8744 - val_loss: 1.2190 - val_acc: 0.6672
Epoch 6/10
100/100 [==============================] - 230s 2s/step - loss: 0.4158 - acc: 0.8866 - val_loss: 1.1834 - val_acc: 0.6850
Epoch 7/10
100/100 [==============================] - 218s 2s/step - loss: 0.3190 - acc: 0.9134 - val_loss: 1.1699 - val_acc: 0.6782
Epoch 8/10
100/100 [==============================] - 219s 2s/step - loss: 0.253

In [82]:
super_model.fit_generator(train_generator, steps_per_epoch=100, epochs=4, validation_data=valid_generator, 
                          validation_steps=valid_full_path_steps, max_queue_size=30, workers=3, 
                          use_multiprocessing=True)

Epoch 1/4
100/100 [==============================] - 239s 2s/step - loss: 0.2194 - acc: 0.9397 - val_loss: 0.9460 - val_acc: 0.7454
Epoch 2/4
100/100 [==============================] - 229s 2s/step - loss: 0.2146 - acc: 0.9422 - val_loss: 0.8309 - val_acc: 0.7690
Epoch 3/4
100/100 [==============================] - 228s 2s/step - loss: 0.1662 - acc: 0.9528 - val_loss: 0.8879 - val_acc: 0.7587
Epoch 4/4
100/100 [==============================] - 230s 2s/step - loss: 0.1782 - acc: 0.9528 - val_loss: 0.9305 - val_acc: 0.7568


In [83]:
preds = []
for i in tqdm_notebook(range(10)):
    test_val_gen = get_train_generator(X_test, y_test, batch_size=BATCH_SIZE, circle=False, shuffle_data=False,
                                       shuffle_frame=True)
    prediction = super_model.predict_generator(test_val_gen, steps=valid_full_path_steps)
    preds.append(prediction)

In [84]:
final_preds = np.zeros_like(preds[0])
for prd in preds:
    final_preds += prd
final_preds /= len(preds)
total_prediction = np.argmax(final_preds, axis=1)
total_prediction.shape

(2366,)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(total_prediction, y_test.argmax(axis=1))

0.8250211327134404

In [88]:
vid_names = os.listdir('data/test_converted/imgs/')
test_full_pass_steps = np.ceil(len(vid_names) / BATCH_SIZE)
preds = []
for i in tqdm_notebook(range(10)):
    test_eval_gen = get_eval_generator(vid_names, batch_size=BATCH_SIZE)
    prediction = super_model.predict_generator(test_eval_gen, steps=test_full_pass_steps, max_queue_size=30, verbose=1)
    preds.append(prediction)
    
final_preds = np.zeros_like(preds[0])
for prd in preds:
    final_preds += prd
final_preds /= len(preds)
total_prediction = np.argmax(final_preds, axis=1)
total_prediction.shape

117/117 [==============================] - 171s 1s/step



(3729,)

In [92]:
sub = pd.DataFrame({'filenames':['{}.avi'.format(name) for name in vid_names], 'classnum':total_prediction})
sub.head()

,filenames,classnum
0,0690.avi,13
1,2654.avi,35
2,3132.avi,75
3,0931.avi,78
4,3689.avi,25


In [93]:
sub.to_csv('rgb_lstm512_pretty_tuned.csv', index=False)

In [86]:
model.save('rgb_lstm512_pretty_tuned.h5')

In [58]:
flow_super_model, flow_model = get_rgb_model(512)

In [61]:
train_generator = get_train_flow_generator(X_train, y_train, batch_size=BATCH_SIZE, circle=True, shuffle_data=True, shuffle_frame=True)
valid_generator = get_train_flow_generator(X_test, y_test, batch_size=BATCH_SIZE, circle=True, shuffle_data=False, shuffle_frame=False)

In [62]:
history = flow_super_model.fit_generator(train_generator, steps_per_epoch=100, epochs=10, validation_data=valid_generator, 
                          validation_steps=valid_full_path_steps, max_queue_size=60, workers=6, 
                          use_multiprocessing=True)

Epoch 1/10


/home/ya-philya/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


100/100 [==============================] - 264s 3s/step - loss: 2.3844 - acc: 0.5019 - val_loss: 4.9726 - val_acc: 0.0465
Epoch 2/10
100/100 [==============================] - 224s 2s/step - loss: 1.5536 - acc: 0.6950 - val_loss: 4.8223 - val_acc: 0.0617
Epoch 3/10
100/100 [==============================] - 225s 2s/step - loss: 1.3149 - acc: 0.7469 - val_loss: 4.5843 - val_acc: 0.0828
Epoch 4/10
100/100 [==============================] - 219s 2s/step - loss: 1.2756 - acc: 0.7531 - val_loss: 4.3778 - val_acc: 0.1090
Epoch 5/10
100/100 [==============================] - 225s 2s/step - loss: 1.1783 - acc: 0.7666 - val_loss: 4.3020 - val_acc: 0.1098
Epoch 6/10
100/100 [==============================] - 225s 2s/step - loss: 1.0905 - acc: 0.7866 - val_loss: 4.5021 - val_acc: 0.1216
Epoch 8/10
100/100 [==============================] - 225s 2s/step - loss: 1.0361 - acc: 0.7950 - val_loss: 4.1698 - val_acc: 0.1191
Epoch 9/10
100/100 [==============================] - 226s 2s/step - loss: 0.968

In [64]:
history = flow_super_model.fit_generator(train_generator, steps_per_epoch=200, epochs=10, validation_data=valid_generator, 
                          validation_steps=valid_full_path_steps, max_queue_size=30, workers=3, 
                          use_multiprocessing=True)

Epoch 1/10
200/200 [==============================] - 341s 2s/step - loss: 1.4077 - acc: 0.6923 - val_loss: 3.7858 - val_acc: 0.1723
Epoch 2/10
200/200 [==============================] - 342s 2s/step - loss: 1.1732 - acc: 0.7413 - val_loss: 3.5515 - val_acc: 0.2078
Epoch 3/10
200/200 [==============================] - 339s 2s/step - loss: 1.1866 - acc: 0.7334 - val_loss: 3.6405 - val_acc: 0.1901
Epoch 4/10
200/200 [==============================] - 342s 2s/step - loss: 1.2532 - acc: 0.7192 - val_loss: 3.7363 - val_acc: 0.1900
Epoch 5/10
200/200 [==============================] - 333s 2s/step - loss: 1.1597 - acc: 0.7334 - val_loss: 3.2765 - val_acc: 0.2454
Epoch 6/10
200/200 [==============================] - 343s 2s/step - loss: 1.1419 - acc: 0.7408 - val_loss: 3.4091 - val_acc: 0.2384
Epoch 7/10
200/200 [==============================] - 330s 2s/step - loss: 1.0719 - acc: 0.7544 - val_loss: 3.3819 - val_acc: 0.2365
Epoch 8/10
200/200 [==============================] - 344s 2s/step - 

In [68]:
for i, l in enumerate(flow_model.layers[1].layer.layers):
    print(i, l.name, l.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d_1 False
13 block2_pool False
14 batch_normalization_1 False
15 add_1 False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_2 False
23 block3_pool False
24 batch_normalization_2 False
25 add_2 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_3 False
33 block4_pool False
34 batch_normalization_3 False
35 add_3 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2

In [73]:
def activ_model(model):
    for l in model.layers[1].layer.layers[126:]:
        l.trainable = True

def disactiv_model(model):
    for l in model.layers[1].layer.layers[126:]:
        l.trainable = False

In [74]:
activ_model(flow_model)
flow_super_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [75]:
history = flow_super_model.fit_generator(train_generator, steps_per_epoch=200, epochs=3, validation_data=valid_generator, 
                          validation_steps=valid_full_path_steps, max_queue_size=30, workers=3, 
                          use_multiprocessing=True)

Epoch 1/3


/home/ya-philya/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


200/200 [==============================] - 346s 2s/step - loss: 4.5091 - acc: 0.0584 - val_loss: 4.8967 - val_acc: 0.0051
Epoch 2/3
200/200 [==============================] - 344s 2s/step - loss: 4.3244 - acc: 0.0666 - val_loss: 4.6854 - val_acc: 0.0228
Epoch 3/3
200/200 [==============================] - 345s 2s/step - loss: 4.0863 - acc: 0.0894 - val_loss: 4.7102 - val_acc: 0.0406


In [77]:
for l in model.layers[1].layer.layers:
    print(l.name, l.trainable)

input_1 False
block1_conv1 False
block1_conv1_bn False
block1_conv1_act False
block1_conv2 False
block1_conv2_bn False
block1_conv2_act False
block2_sepconv1 False
block2_sepconv1_bn False
block2_sepconv2_act False
block2_sepconv2 False
block2_sepconv2_bn False
conv2d_1 False
block2_pool False
batch_normalization_1 False
add_1 False
block3_sepconv1_act False
block3_sepconv1 False
block3_sepconv1_bn False
block3_sepconv2_act False
block3_sepconv2 False
block3_sepconv2_bn False
conv2d_2 False
block3_pool False
batch_normalization_2 False
add_2 False
block4_sepconv1_act False
block4_sepconv1 False
block4_sepconv1_bn False
block4_sepconv2_act False
block4_sepconv2 False
block4_sepconv2_bn False
conv2d_3 False
block4_pool False
batch_normalization_3 False
add_3 False
block5_sepconv1_act False
block5_sepconv1 False
block5_sepconv1_bn False
block5_sepconv2_act False
block5_sepconv2 False
block5_sepconv2_bn False
block5_sepconv3_act False
block5_sepconv3 False
block5_sepconv3_bn False
add_4 Fa

In [15]:
preds = {}

for name in tqdm_notebook(os.listdir('data/test_converted/imgs/')):
    X_batch = np.zeros((1, 10, 240, 320, 3))
    for k in range(10):
        img = cv2.imread('data/test_converted/imgs/{}/{}.jpg'.format(name, k))
        X_batch[0, k] = preprocess_input(img)
    prediction = model.predict_on_batch(X_batch)[0]
    preds[name] = prediction

In [18]:
test_fn = []
test_class = []
for fn, cls in preds.items():
    test_fn += [fn+'.avi']
    test_class += [cls.argmax()]
sub = pd.DataFrame({'filename':test_fn, 'classnum':test_class})
sub.head()

,filename,classnum
0,0690.avi,13
1,2654.avi,35
2,3132.avi,48
3,0931.avi,15
4,3689.avi,25


In [19]:
sub.to_csv('rgb_lstm_model1_sub.csv',index=False)

In [30]:
model.save('rgb_lstm_model1.h5')